In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
import feature_engine
import sklearn
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from feature_engine.imputation import MeanMedianImputer
import math


In [17]:
excel_file_path = 'budgetusd.xlsx'

sheet_name = "Revenue"

df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

# Load the Excel file into a pandas DataFrame

df

,Date,Percentage %,Holidays Local,Winter North,Holidays North,Local Rainy Season,International Flights,Marketing,Resturant Revenue,Bar Revenue,Rooms Revenue,Total Revenue
0,2022-01,0.61,3,Yes,7,No,1,62.111111,7771.638889,3972.555556,13136.888889,24881.083333
1,2022-02,0.62,1,Yes,0,No,1,62.111111,5660.000000,3773.333333,11430.527778,20863.861111
2,2022-03,0.39,0,Yes,0,No,1,99.305556,5299.861111,3533.222222,9742.611111,18575.694444
3,2022-04,0.70,5,No,0,No,1,246.055556,11300.750000,7533.833333,15043.805556,33878.388889
4,2022-05,0.47,1,No,0,Medium,1,144.920389,5555.010000,3703.340000,8546.169167,17804.519167
5,2022-06,0.33,0,No,0,Medium,1,118.458222,2821.991944,1881.328056,7171.419167,11874.739167
6,2022-07,0.67,3,No,20,Medium,1,46.761375,6345.989444,4230.659722,15864.973889,26441.623056
7,2022-08,0.49,2,No,10,Medium,1,149.967917,5398.003139,3598.668611,11925.820833,20922.492583
8,2022-09,0.51,3,No,0,Yes,1,170.240833,5495.210667,3663.473778,13738.026667,22896.711111
9,2022-10,0.40,0,No,0,Yes,1,176.074631,3969.148564,2646.099042,7765.725450,14380.973056


In [18]:
# This function will remove decimal points

def round_up_numeric(value):
    if pd.notnull(value) and isinstance(value, (int, float)):
        return math.ceil(value * 100) / 100
    return value

df =  df.applymap(round_up_numeric)


In [19]:
df

,Date,Percentage %,Holidays Local,Winter North,Holidays North,Local Rainy Season,International Flights,Marketing,Resturant Revenue,Bar Revenue,Rooms Revenue,Total Revenue
0,2022-01,0.61,3.0,Yes,7.0,No,1.0,62.12,7771.64,3972.56,13136.89,24881.09
1,2022-02,0.62,1.0,Yes,0.0,No,1.0,62.12,5660.00,3773.34,11430.53,20863.87
2,2022-03,0.39,0.0,Yes,0.0,No,1.0,99.31,5299.87,3533.23,9742.62,18575.70
3,2022-04,0.70,5.0,No,0.0,No,1.0,246.06,11300.75,7533.84,15043.81,33878.39
4,2022-05,0.47,1.0,No,0.0,Medium,1.0,144.93,5555.01,3703.34,8546.17,17804.52
5,2022-06,0.33,0.0,No,0.0,Medium,1.0,118.46,2822.00,1881.33,7171.42,11874.74
6,2022-07,0.67,3.0,No,20.0,Medium,1.0,46.77,6345.99,4230.66,15864.98,26441.63
7,2022-08,0.49,2.0,No,10.0,Medium,1.0,149.97,5398.01,3598.67,11925.83,20922.50
8,2022-09,0.51,3.0,No,0.0,Yes,1.0,170.25,5495.22,3663.48,13738.03,22896.72
9,2022-10,0.40,0.0,No,0.0,Yes,1.0,176.08,3969.15,2646.10,7765.73,14380.98
